In [ ]:
# test finBERT outputs

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [4]:
tickers = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'MSFT', 'IBM', 'ORCL', 'NVDA', 'INTC']
original_data = "dataset/news"
new_data = "dataset/polygon_title_sentiment"
start_time = "2024-07-01"
end_time = "2024-09-01"

In [18]:
merged_dfs = []
for ticker in tickers:
#     old_df_news = pd.read_csv(f"{original_data}/{ticker}_main.csv")
    old_df_rela = pd.read_csv(f"{original_data}/{ticker}_relation.csv")
    new_df = pd.read_csv(f"{new_data}/{ticker}_title_sentiments.csv")
    filtered_rela = old_df_rela[old_df_rela['ticker'] == old_df_rela['source_ticker']]
#     print(new_df)
#     print(filtered_rela)
    merged_df = pd.merge(new_df, filtered_rela, on="news_id", how="left")
    # print(merged_df)
#     merged_df = merged_df[(merged_df["time_x"] >= start_time) & (merged_df["time_x"] <= end_time)]
    merged_df = merged_df.dropna()
    # print(merged_df)
    correct_val = len(merged_df[merged_df["sentiment_x"] == merged_df["sentiment_y"]])
    total_val = len(merged_df)
    print(f"{ticker}: {correct_val} / {total_val}")
    merged_dfs.append(merged_df)

META: 178 / 379
AAPL: 321 / 663
AMZN: 344 / 767
NFLX: 81 / 147
GOOGL: 288 / 671
MSFT: 343 / 722
IBM: 104 / 150
ORCL: 98 / 145
NVDA: 631 / 1272
INTC: 136 / 268


In [ ]:
print(merged_dfs[0])

Below are for a separate test not for title data

In [3]:
df_news = pd.read_csv('dataset/news/AAPL_main.csv')
df_rela = pd.read_csv('dataset/AAPL_rel_2.csv')

In [4]:
df_aapl = pd.read_csv('dataset/stocks/AAPL.csv')

In [5]:
df_news_t = df_rela[df_rela['ticker'] == df_rela['source_ticker']]
df_news_t['time'] = pd.to_datetime(df_news_t['time']).dt.tz_localize("UTC") \
                    .dt.tz_convert("America/New_York")
df_news_t['sentiment'] = df_news_t['sentiment'].map(
    {'neutral':0, 'positive':1, 'negative':-4.22} # maybe
)
df_news_t['date'] = df_news_t['time'].dt.date

In [6]:
s_senti = df_news_t.groupby('date')['sentiment'].sum().reset_index()

In [7]:
df_aapl_t = df_aapl[df_aapl['time'] >= "2024-07-02"]
df_aapl_t['time'] = pd.to_datetime(df_aapl_t['time']).dt.tz_localize("UTC") \
                    .dt.tz_convert("America/New_York")
df_aapl_t['change'] = df_aapl['close'].diff()
df_aapl_t['date'] = df_aapl_t['time'].dt.date

/var/folders/0q/kv5wlvmd4wdc2_bz7dkwlj180000gn/T/ipykernel_51877/3429688966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aapl_t['time'] = pd.to_datetime(df_aapl_t['time']).dt.tz_localize("UTC") \
/var/folders/0q/kv5wlvmd4wdc2_bz7dkwlj180000gn/T/ipykernel_51877/3429688966.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aapl_t['change'] = df_aapl['close'].diff()
/var/folders/0q/kv5wlvmd4wdc2_bz7dkwlj180000gn/T/ipykernel_51877/3429688966.py:5: SettingWithCopyWarning: 
A value is trying to be s

In [8]:
s_stock = df_aapl_t[['date', 'change']].reset_index(drop=True)

In [9]:
s_corr = pd.merge(s_stock, s_senti, on="date", how="left")
s_corr['sentiment'] = s_corr['sentiment'].fillna(0)

In [10]:
stats.pearsonr(s_corr['change'], s_corr['sentiment'])

PearsonRResult(statistic=0.06728981144642374, pvalue=0.5430744928395074)

In [11]:
split_index = int(len(s_corr) * 0.8)
s_corr_train = s_corr.iloc[:split_index] 
s_corr_test = s_corr.iloc[split_index:]

In [12]:
model = LinearRegression()
model.fit(s_corr_train[['sentiment']], s_corr_train['change'])
model.coef_[0], model.intercept_

(0.0572883252716194, 0.20694630541925169)